In [ ]:
import os
import json
import numpy as np
import itertools
import math

input_folder = "20node"
output_folder = "output_select_path_greedy"
os.makedirs(output_folder, exist_ok=True)

# --------------------------
# HÀM TÍNH VS
# --------------------------
def compute_vs(p1, p2, v_f, v_AUV):
    """Tính vận tốc tổng hợp v_s giữa 2 vị trí p1, p2"""
    x1, y1, z1 = p1
    x2, y2, z2 = p2

    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV

    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)

    # tính góc và vận tốc hiệu dụng
    if abs(cos_beta) < 1e-6:
        cos_beta = 1e-6

    inner = (v_f * cos_beta) / v_AUV
    inner = np.clip(inner, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta)
    return v_s


# --------------------------
# HÀM TÍNH THỜI GIAN DI CHUYỂN
# --------------------------
def travel_time(path, positions, v_f, v_AUV):
    """Tính tổng thời gian di chuyển của AUV theo 1 đường đi cụ thể"""
    total_time = 0.0
    O = np.array([0, 0, 0])

    # từ O đến node đầu tiên
    p1 = O
    p2 = positions[path[0]]
    d = np.linalg.norm(p2 - p1)
    v_s = compute_vs(p1, p2, v_f, v_AUV)
    total_time += d / v_s

    # giữa các cluster head
    for i in range(len(path) - 1):
        p1 = positions[path[i]]
        p2 = positions[path[i + 1]]
        d = np.linalg.norm(p2 - p1)
        v_s = compute_vs(p1, p2, v_f, v_AUV)
        total_time += d / v_s

    # quay về O
    p1 = positions[path[-1]]
    p2 = O
    d = np.linalg.norm(p2 - p1)
    v_s = compute_vs(p1, p2, v_f, v_AUV)
    total_time += d / v_s

    return total_time




def path_selection(cluster_heads, positions, v_f=1.2, v_AUV=3.0):
    n = len(cluster_heads)
    best_time = float('inf')
    best_path = None
    for path in itertools.permutations(cluster_heads):
        total_time = travel_time(path, positions, v_f, v_AUV)
        if total_time < best_time:
            best_time = total_time
            best_path = path
    return best_path, best_time


# =============================
# TÍNH NĂNG LƯỢNG
# =============================
def compute_energy(best_time, G=100, L=1024, n=4,
                   P_t=1.6e-3, P_r=0.8e-3, P_idle=0.1e-3,
                   DR=4000, DR_i=1e6):
    """Tính năng lượng tiêu thụ cho Member Node và Target Node"""
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN

    E_rx_TN = G * P_r * L * n / DR
    E_tx_TN = G * P_t * L * n / DR_i
    E_idle_TN = (best_time - (G * L * n / DR) - (G * L * n / DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN

    return {
        "Member": {"E_tx": E_tx_MN, "E_idle": E_idle_MN, "E_total": E_total_MN},
        "Target": {"E_rx": E_rx_TN, "E_tx": E_tx_TN, "E_idle": E_idle_TN, "E_total": E_total_TN}
    }

for filename in os.listdir(input_folder):
    if not filename.endswith(".json"):
        continue

    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, f"result_{filename}")

    print(f"\nĐang xử lý {filename}")

    with open(input_path, "r", encoding="utf-8") as f:
        nodes_data = json.load(f)

    cluster_heads = []
    positions = {}
    for cid, info in nodes_data.items():
        ch = info["cluster_head"]
        cluster_heads.append(ch)
        positions[ch] = np.array(info["center"])

    # Tìm đường đi tối ưu theo GREEDY với tính Vs
    best_path, best_time = path_selection(cluster_heads, positions, v_f=1.2, v_AUV=3.0)

    result = {
        "input_file": filename,
        "cluster_heads": cluster_heads,
        "best_path": list(best_path),
        "total_time": round(float(best_time), 4)
    }

    with open(output_path, "w", encoding="utf-8") as f_out:
        json.dump(result, f_out, ensure_ascii=False, indent=4)

    print(f"Kết quả của {output_path}:")
    print("Đường đi tối ưu:", " -> ".join(map(str, best_path)))
    print(f"Tổng thời gian di chuyển: {best_time:.4f} s")





Đang xử lý nodes_20.json
Kết quả của output_select_path_greedy\result_nodes_20.json:
Đường đi tối ưu: 10 -> 7 -> 18 -> 12 -> 13 -> 4
Tổng thời gian di chuyển: 84.6125 s


In [ ]:
import os
import json
import numpy as np
import itertools
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime


# --------------------------
# HÀM TÍNH VS
# --------------------------
def compute_vs(p1, p2, v_f, v_AUV):
    """Tính vận tốc tổng hợp v_s giữa 2 vị trí p1, p2"""
    x1, y1, z1 = p1
    x2, y2, z2 = p2

    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV

    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)

    # tính góc và vận tốc hiệu dụng
    if abs(cos_beta) < 1e-6:
        cos_beta = 1e-6

    inner = (v_f * cos_beta) / v_AUV
    inner = np.clip(inner, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta)
    return v_s


# --------------------------
# HÀM TÍNH THỜI GIAN DI CHUYỂN
# --------------------------
def travel_time(path, positions, v_f, v_AUV):
    """Tính tổng thời gian di chuyển của AUV theo 1 đường đi cụ thể"""
    total_time = 0.0
    O = np.array([0, 0, 0])

    # từ O đến node đầu tiên
    p1 = O
    p2 = positions[path[0]]
    d = np.linalg.norm(p2 - p1)
    v_s = compute_vs(p1, p2, v_f, v_AUV)
    total_time += d / v_s

    # giữa các cluster head
    for i in range(len(path) - 1):
        p1 = positions[path[i]]
        p2 = positions[path[i + 1]]
        d = np.linalg.norm(p2 - p1)
        v_s = compute_vs(p1, p2, v_f, v_AUV)
        total_time += d / v_s

    # quay về O
    p1 = positions[path[-1]]
    p2 = O
    d = np.linalg.norm(p2 - p1)
    v_s = compute_vs(p1, p2, v_f, v_AUV)
    total_time += d / v_s

    return total_time

def path_selection(cluster_heads, positions, v_f=1.2, v_AUV=3.0):
    n = len(cluster_heads)
    best_time = float('inf')
    best_path = None
    for path in itertools.permutations(cluster_heads):
        total_time = travel_time(path, positions, v_f, v_AUV)
        if total_time < best_time:
            best_time = total_time
            best_path = path
    return best_path, best_time


# =============================
# TÍNH NĂNG LƯỢNG 
# =============================
def compute_energy(best_time, G=100, L=1024, n=4,
                   P_t=1.6e-3, P_r=0.8e-3, P_idle=0.1e-3,
                   DR=4000, DR_i=1e6):
    """Tính năng lượng tiêu thụ cho Member Node và Target Node"""
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN

    E_rx_TN = G * P_r * L * n / DR
    E_tx_TN = G * P_t * L * n / DR_i
    E_idle_TN = (best_time - (G * L * n / DR) - (G * L * n / DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN

    return {
        "Member": {"E_tx": E_tx_MN, "E_idle": E_idle_MN, "E_total": E_total_MN},
        "Target": {"E_rx": E_rx_TN, "E_tx": E_tx_TN, "E_idle": E_idle_TN, "E_total": E_total_TN}
    }


# =============================
# VẼ ĐƯỜNG ĐI 3D
# =============================
def plot_path_3d(coords, path, title="Best path", save_path=None):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection="3d")
    path_coords = coords[path + [path[0]]]
    ax.plot(path_coords[:, 0], path_coords[:, 1], path_coords[:, 2], "-o")
    ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c="red", s=50)
    for i, (x, y, z) in enumerate(coords):
        ax.text(x, y, z, str(i))
    ax.set_title(title)
    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()


# =============================
# LƯU KẾT QUẢ THEO CHUẨN PSO
# =============================
def save_results(output_dir, filename, best_path, best_time, energy):
    result = {
        "best_path": list(best_path),
        "best_time": best_time,
        "energy": energy,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, filename.replace(".json", "_result.json"))
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(result, f, indent=4, ensure_ascii=False)
    print(f"Lưu kết quả vào: {output_path}")


# =============================
# CHẠY GREEDY TRÊN TOÀN BỘ FILE
# =============================
if __name__ == "__main__":
    input_dir = "D:/Year 4/tiến hóa/project/20node/"
    output_dir = "D:/Year 4/tiến hóa/project/UWSN_greedy/output_path/output_greedy/"

    json_files = [f for f in os.listdir(input_dir) if f.endswith(".json")]
    print(f"Tìm thấy {len(json_files)} file JSON trong thư mục input.")

    for file in json_files:
        input_path = os.path.join(input_dir, file)
        print(f"\n=== Xử lý file: {file} ===")

        with open(input_path, "r", encoding="utf-8") as f:
            nodes_data = json.load(f)

        # Lấy cluster head & tọa độ
        cluster_heads = [info["cluster_head"] for info in nodes_data.values()]
        positions = {info["cluster_head"]: np.array(info["center"]) for info in nodes_data.values()}
        coords = np.array([info["center"] for info in nodes_data.values()])

        # Giữ nguyên thuật toán Greedy gốc
        best_path, best_time = path_selection(cluster_heads, positions, v_f=1.2, v_AUV=3.0)

        print(f"Best path: {best_path}")
        print(f"Total travel time: {best_time:.4f}s")

        # Tính năng lượng & lưu kết quả
        energy = compute_energy(best_time)
        save_results(output_dir, file, best_path, best_time, energy)

        # Vẽ và lưu biểu đồ 3D
        """plot_path_3d(coords, best_path,
                     title=f"{file} - Min Time ({best_time:.2f}s)",
                     save_path=os.path.join(output_dir, file.replace(".json", "_path.png")))"""

    


Tìm thấy 2 file JSON trong thư mục input.

=== Xử lý file: nodes_100.json ===
